In [7]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from databroker import DataBroker as db
from sixtools.rixs_wrapper import make_scan, centroids_to_spectrum, make_dataset
from sixtools.tests.test_sixtools import make_test_dataset
from IPython.display import clear_output, display

%matplotlib widget

In [43]:
%%capture
fig0, ax0 = plt.subplots(num=0, clear=True)
fig1, ax1 = plt.subplots(num=1, clear=True)
fig2, ax2 = plt.subplots(num=2, clear=True)
fig3, ax3 = plt.subplots(num=3, clear=True)

## Define and execute processing

In [2]:
scan_id = 51402
process_dicts_high_2theta = {'light_ROI': [800, 1230, 175, 1500],
                                 'curvature': np.array([0., 0., 0.]),
                                 'bins': 0.5,
                                 'background': None}

ds = make_dataset(db[scan_id], **process_dicts_high_2theta)

## Plot it

In [12]:
fig0, ax0 = plt.subplots(num=0, clear=True)

colors = iter(plt.cm.viridis(np.linspace(0, 1, ds['intensity'].sum('y').size)))
for event_val, event in ds.groupby(ds.attrs['event_name']):
    for frame_val, frame in event.groupby('frame'):
        ax0.errorbar(frame['pixel'].values, frame['intensity'].values, frame['error'].values, color=next(colors),
                    fmt='.-',
                label="{}={} #{}".format(ds.attrs['event_name'], event_val, frame_val))

ax0.set_xlabel('pixel')
ax0.set_ylabel('I')
display(fig0.canvas)

FigureCanvasNbAgg()

# Make fake spectrum

In [13]:
ds_fake = make_test_dataset()

fig1, ax1 = plt.subplots(num=1, clear=True)

colors = iter(plt.cm.viridis(np.linspace(0, 1, ds_fake['intensity'].sum('y').size)))
for event_val, event in ds_fake.groupby(ds_fake.attrs['event_name']):
    for frame_val, frame in event.groupby('frame'):
        ax1.errorbar(frame['pixel'].values, frame['intensity'].values, frame['error'].values, color=next(colors),
                    fmt='.-',
                label="{}={} #{}".format(ds_fake.attrs['event_name'], event_val, frame_val))

ax1.set_xlabel('pixel')
ax1.set_ylabel('I')
display(fig1.canvas)

FigureCanvasNbAgg()

## Offset values based on known curvature

In [19]:
ds_fake = make_test_dataset()
center_offsets = xr.DataArray(ds_fake.attrs['centers']-500, dims='centers')

ds_fake_offset = ds_fake.copy(deep=True)
ds_fake_offset['pixel'] = ds_fake['pixel'] - center_offsets

fig2, ax2 = plt.subplots(num=2, clear=True)

colors = iter(plt.cm.viridis(np.linspace(0, 1, ds_fake_offset['intensity'].sum('y').size)))
for event_val, event in ds_fake_offset.groupby(ds_fake_offset.attrs['event_name']):
    for frame_val, frame in event.groupby('frame'):
        ax2.errorbar(frame['pixel'].values, frame['intensity'].values, frame['error'].values, color=next(colors),
                    fmt='.-',
                label="{}={} #{}".format(ds_fake.attrs['event_name'], event_val, frame_val))

ax2.set_xlabel('pixel')
ax2.set_ylabel('I')
display(fig2.canvas)

FigureCanvasNbAgg()

In [50]:
bins = ds_fake_offset.attrs['bins']
y = ds_fake_offset.attrs['y']
I, _ = np.histogram(ds_fake_offset['pixel'].values, bins=bins, weights=ds_fake_offset['intensity'])

fig3, ax3 = plt.subplots(num=3, clear=True)
ax3.plot(y, I)

display(fig3.canvas)

FigureCanvasNbAgg()